# NEON

This notebook is an excercise in executing ISOFIT on two dates from the NEON dataset and interpreting the outputs of ISOFIT. 

In [1]:
# Imports and logging
import logging
import os

logging.getLogger().setLevel(logging.INFO)

# Setup

The NEON dataset is structured as the following:
```
├── FieldSpectrometer
│   ├── BlackTarp01
│   │   ├── Data
│   │   │   ├── BlackTarp01_IndScienceSpectra.dat
│   │   │   ├── BlackTarp01_PanelBRDF.dat
│   │   │   ├── BlackTarp01_PanelSpectra.dat
│   │   │   ├── BlackTarp01_RadTrace.dat
│   │   │   ├── BlackTarp01_Refl.dat
│   │   │   ├── BlackTarp01_ScienceSpectra.dat
│   │   │   └── BlackTarp01_optNum.dat
│   │   └── Plot
│   │       ├── 01_BlackTarp01_optNum.png
│   │       ├── 02_BlackTarp01_RadTrace.png
│   │       ├── 03_BlackTarp01_PanelBRDF.png
│   │       ├── 04_BlackTarp01_PanelSpectra.png
│   │       ├── 05_BlackTarp01_ScienceSpectra.png
│   │       └── 06_BlackTarp01_Refl.png
│   ├── BlackTarp02
│   │   └── ...
│   ├── BlackTarp03
│   │   └── ...
│   ├── RoadEW01
│   │   └── ...
│   ├── RoadEW02
│   │   └── ...
│   ├── RoadEW03
│   │   └── ...
│   ├── RoadNS01
│   │   └── ...
│   ├── RoadNS02
│   │   └── ...
│   ├── RoadNS03
│   │   └── ...
│   ├── Veg01
│   │   └── ...
│   ├── Veg02
│   │   └── ...
│   ├── Veg03
│   │   └── ...
│   ├── WhiteTarp01
│   │   └── ...
│   ├── WhiteTarp02
│   │   └── ...
│   └── WhiteTarp03
│   │   └── ...
├── Radiance
│   ├── NIS01_20210403_173647_obs_ort
│   ├── NIS01_20210403_173647_obs_ort.hdr
│   ├── NIS01_20210403_173647_rdn_ort
│   ├── NIS01_20210403_173647_rdn_ort.hdr
│   ├── NIS01_20210403_174150_obs_ort
│   ├── NIS01_20210403_174150_obs_ort.hdr
│   ├── NIS01_20210403_174150_rdn_ort
│   └── NIS01_20210403_174150_rdn_ort.hdr
├── Reflectance
│   ├── NIS01_20210403_173647_rdn_ort_atm_nodata.bsq
│   ├── NIS01_20210403_173647_rdn_ort_atm_nodata.hdr
│   ├── NIS01_20210403_174150_rdn_ort_atm_nodata.bsq
│   └── NIS01_20210403_174150_rdn_ort_atm_nodata.hdr
└── Report
    └── 20210403_R10E_P1C2_TBMT_VicariousCalibration_Report.pdf
```

ISOFIT needs at minimum three pieces as input:

    1. Radiance measurements (rdn)
    2. Observation values    (obs)
    3. Location information  (loc)

NEON provides the RDN and OBS files for two dates:

```
Radiance
├── 173647
│   ├── NIS01_20210403_173647_obs_ort
│   ├── NIS01_20210403_173647_obs_ort.hdr
│   ├── NIS01_20210403_173647_rdn_ort
│   └── NIS01_20210403_173647_rdn_ort.hdr
└── 174150
    ├── NIS01_20210403_174150_obs_ort
    ├── NIS01_20210403_174150_obs_ort.hdr
    ├── NIS01_20210403_174150_rdn_ort
    └── NIS01_20210403_174150_rdn_ort.hdr
```

As such, the LOC file needs to be generated or faked. 


# Apply OE

The next part walks through running the ISOFIT utility script `isofit/utils/apply_oe.py`. This is the first step of executing ISOFIT and will generate a default configuration.

In [2]:
# Insert keys into the env as these are normally retrieved from there
if not os.environ.get('ISOFIT'):
    import isofit

    os.environ['ISOFIT'] = isofit.__path__[0]

# If running inside the Docker, this is already set. You may need to point this to your sRTMnet installation path.
os.environ['EMULATOR_PATH'] # = /path/to/sRTMnet

# This is just to retrieve surface files -- path to your ISOFIT installation
os.environ['ISOFIT'] = '/isofit'

In [3]:
# Path to the input NEON data
input_path = '/isofit/.idea/neon/data/TableMountainCalibration/Radiance'

# Which NEON date to process - change this to process a different date
neon_ids = ['173647', '174150']
neon_id  = neon_ids[0]

# ISOFIT output directory
output_path = '/isofit/.idea/neon/run/output/'

# NEON doesn't provide a surface file, just pull from one of the examples
f"{os.environ['ISOFIT']}/examples/"

# Args for apply_oe since it's not intended to be called as a module
class ARGS:
    input_radiance    = f'{input_path}/NIS01_20210403_{neon_id}_rdn_ort'
    input_loc         = f'{input_path}/NIS01_20210403_{neon_id}_loc_ort'
    input_obs         = f'{input_path}/NIS01_20210403_{neon_id}_obs_ort'
    working_directory = output_path
    sensor            = f'neon'
    emulator_base     = os.environ['EMULATOR_PATH']
    logging_level     = 'INFO'
    n_cores           = os.cpu_count()
    segmentation_size = 40
    surface_category  = 'multicomponent_surface'

    # Allows apply_oe to not raise an exception on missing parameters
    def __getattr__(self, key):
        if hasattr(ARGS, key):
            return getattr(ARGS, key)
    # int params can't get around config type checks
    presolve           = 0
    empirical_line     = False
    analytical_line    = False
    segmentation_size  = 40
    pressure_elevation = 0
    # Do not change this
    ray_temp_dir = '/tmp/ray'

args = ARGS()

In [4]:
# Generate the initial configuration and inversions
from isofit.utils.apply_oe import apply_oe

apply_oe(args)

INFO:2023-06-07,15:30:18 ||| <__main__.ARGS object at 0xffff8bdf0910>
INFO:2023-06-07,15:30:19 ||| No surface model defined
INFO:2023-06-07,15:30:19 ||| no noise path found, proceeding without
    3.19215245] span 1.0 quadrants, while data spans 1.0 quadrants
INFO:2023-06-07,15:31:02 ||| Observation means:
INFO:2023-06-07,15:31:02 ||| Path (km): 1.037891115734179
INFO:2023-06-07,15:31:02 ||| To-sensor Zenith (deg): 171.20155848406853
INFO:2023-06-07,15:31:02 ||| To-sensor Azimuth (deg): 191.0493887892825
INFO:2023-06-07,15:31:02 ||| Altitude (km): 1.0273667760463228
INFO:2023-06-07,15:31:02 ||| Full (non-aerosol) LUTs:
INFO:2023-06-07,15:31:02 ||| Elevation: None
INFO:2023-06-07,15:31:02 ||| To-sensor azimuth: [  3.19215245  95.77536366 107.72859317 186.28088126 257.15695548
 268.7331669 ]
INFO:2023-06-07,15:31:02 ||| To-sensor zenith: [160.4236 170.2112 179.9987]
INFO:2023-06-07,15:31:02 ||| H2O Vapor: [0.05   0.2975 0.545  0.7925 1.04   1.2875 1.535  1.7825 2.03   2.2775
 2.525  2.77

RAYARGS: {'address': None, '_redis_password': None, '_temp_dir': '/tmp/ray', 'ignore_reinit_error': True, 'include_dashboard': False, 'local_mode': False}


2023-06-07 15:31:03,979	INFO worker.py:1553 -- Started a local Ray instance.
INFO:2023-06-07,15:31:04 ||| Building first forward model, will generate any necessary LUTs
INFO:2023-06-07,15:31:08 ||| Prebuilt LUT interpolators found, loading from disk


FileNotFoundError: [Errno 2] No such file or directory: '/isofit/.idea/neon/run/output/data/surface.mat'

(raylet) [2023-06-07 15:31:14,012 E 26467 26503] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-06-07_15-31-02_197185_26300 is over 95% full, available space: 0; capacity: 62671097856. Object creation will fail if spilling is required.
(raylet) [2023-06-07 15:31:24,098 E 26467 26503] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-06-07_15-31-02_197185_26300 is over 95% full, available space: 0; capacity: 62671097856. Object creation will fail if spilling is required.
(raylet) [2023-06-07 15:31:34,149 E 26467 26503] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-06-07_15-31-02_197185_26300 is over 95% full, available space: 0; capacity: 62671097856. Object creation will fail if spilling is required.
(raylet) [2023-06-07 15:31:44,226 E 26467 26503] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-06-07_15-31-02_197185_26300 is over 95% full, available space: 0; capacity: 62671097856. Object creation will fail if spilling is required.
(raylet)

(raylet) [2023-06-07 15:36:35,799 E 26467 26503] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-06-07_15-31-02_197185_26300 is over 95% full, available space: 0; capacity: 62671097856. Object creation will fail if spilling is required.
(raylet) [2023-06-07 15:36:45,897 E 26467 26503] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-06-07_15-31-02_197185_26300 is over 95% full, available space: 0; capacity: 62671097856. Object creation will fail if spilling is required.
(raylet) [2023-06-07 15:36:55,989 E 26467 26503] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-06-07_15-31-02_197185_26300 is over 95% full, available space: 0; capacity: 62671097856. Object creation will fail if spilling is required.
(raylet) [2023-06-07 15:37:05,990 E 26467 26503] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-06-07_15-31-02_197185_26300 is over 95% full, available space: 0; capacity: 62671097856. Object creation will fail if spilling is required.
(raylet)

(raylet) [2023-06-07 15:41:57,841 E 26467 26503] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-06-07_15-31-02_197185_26300 is over 95% full, available space: 0; capacity: 62671097856. Object creation will fail if spilling is required.
(raylet) [2023-06-07 15:42:07,938 E 26467 26503] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-06-07_15-31-02_197185_26300 is over 95% full, available space: 0; capacity: 62671097856. Object creation will fail if spilling is required.
